In [ ]:
import json
import os
from dataclasses import dataclass

from IPython.display import Markdown
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.formrecognizer import DocumentAnalysisClient, AnalysisFeature
from azure.ai.formrecognizer import AnalyzeResult as FormRecognizerAnalyzeResult
from azure.ai.documentintelligence.models import (
    AnalyzeResult,
    ContentFormat,
    AnalyzeDocumentRequest,
    DocumentAnalysisFeature,
)
USE_DOC_INTEL_PREVIEW_VERSION = True
DOC_INTEL_MODEL_ID = "prebuilt-layout" # E.g. "prebuilt-read", "prebuilt-layout", or "prebuilt-document"

# Possible Document Intelligence features
# v4.0 (Preview): ['ocrHighResolution', 'languages', 'barcodes', 'formulas', 'styleFont', 'keyValuePairs', 'queryFields']
# v3.3 (GA):      ['ocrHighResolution', 'languages', 'barcodes', 'formulas', 'styleFont']
DOC_INTEL_FEATURES = ['ocrHighResolution', 'languages', 'styleFont']
# Load environment variables from Function App local settings file
with open("../function_app/local.settings.json", "rb") as f:
    local_settings = json.load(f)
    os.environ.update(local_settings["Values"])

DOC_INTEL_ENDPOINT = os.getenv("DOC_INTEL_ENDPOINT")
DOC_INTEL_API_KEY = os.getenv("DOC_INTEL_API_KEY")

In [ ]:
di_client = DocumentIntelligenceClient(
        endpoint=DOC_INTEL_ENDPOINT, 
        credential=AzureKeyCredential(DOC_INTEL_API_KEY),
        api_version="2024-07-31-preview",
    )
enabled_features = [DocumentAnalysisFeature(feature) for feature in DOC_INTEL_FEATURES]



In [ ]:
import base64

def convert_pdf_to_base64(pdf_path: str):
    # Read the PDF file in binary mode, encode it to base64, and decode to string
    with open(pdf_path, "rb") as file:
        base64_encoded_pdf = base64.b64encode(file.read()).decode()
    return base64_encoded_pdf
analyze_request = AnalyzeDocumentRequest(bytes_source=convert_pdf_to_base64("/home/azureuser/multimodal-ai-llm-processing-accelerator/multimodel_pdf/oral_cancer_range1-6.pdf"))

poller = di_client.begin_analyze_document(
            model_id=DOC_INTEL_MODEL_ID,
            analyze_request=analyze_request,
            output_content_format=ContentFormat.MARKDOWN,
            features=enabled_features
        )
analyzedDocumentResult = poller.result()

In [ ]:
print(analyzedDocumentResult.content)


In [ ]:
Markdown(analyzedDocumentResult.content)